In [1]:
'''
References
https://stackoverflow.com/questions/51448897/in-pytorch-f-nll-loss-expected-object-of-type-torch-longtensor-but-found-type
    
https://stackoverflow.com/questions/59718130/what-are-c-classes-for-a-nllloss-loss-function-in-pytorch
https://hackernoon.com/binary-face-classifier-using-pytorch-2d835ccb7816
https://pytorch.org/docs/master/nn.html#bcewithlogitsloss
    
For binary classification, you can use 1 output unit 
https://stackoverflow.com/questions/53628622/loss-function-its-inputs-for-binary-classification-pytorch
    Note: BCEWithLogicsLoss is BCELoss with sigmoid function
    
# Try one calssification from model
# https://medium.com/@hasithsura/audio-classification-d37a82d6715
'''

'\nReferences\nhttps://stackoverflow.com/questions/51448897/in-pytorch-f-nll-loss-expected-object-of-type-torch-longtensor-but-found-type\n    \nhttps://stackoverflow.com/questions/59718130/what-are-c-classes-for-a-nllloss-loss-function-in-pytorch\nhttps://hackernoon.com/binary-face-classifier-using-pytorch-2d835ccb7816\nhttps://pytorch.org/docs/master/nn.html#bcewithlogitsloss\n    \nFor binary classification, you can use 1 output unit \nhttps://stackoverflow.com/questions/53628622/loss-function-its-inputs-for-binary-classification-pytorch\n    Note: BCEWithLogicsLoss is BCELoss with sigmoid function\n    \n# Try one calssification from model\n# https://medium.com/@hasithsura/audio-classification-d37a82d6715\n'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torchaudio
import pandas as pd
import numpy as np
import librosa 
from sklearn.utils import shuffle #shuffle

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

cpu


In [27]:
csvData = pd.read_csv('JsonContentInCsvmultipleword.csv')
print(csvData.iloc[0, :])
batch_sz = 4 # batch size

key         data/F/F01/session1/wav_headMic/0001.wav
duration                                     10.2168
text                       [say Ah-P-Eee repeatedly]
label                                              1
Name: 0, dtype: object


In [4]:
file_names = []
labels = []


for i in range(0,len(csvData)):
    file_names.append(csvData.iloc[i, 0]) # First column is key which is path of audio file
    labels.append(csvData.iloc[i, 3]) # 3rd column (starting at 0) is label
file_names
1

1

In [28]:
def get_melspectrogram_db(file_path, sr=None, n_fft=2048, hop_length=512, n_mels=128, fmin=20, fmax=8300, top_db=80):
  wav,sr = librosa.load(file_path,sr=sr)
  if wav.shape[0]<5*sr:
    wav=np.pad(wav,int(np.ceil((5*sr-wav.shape[0])/2)),mode='reflect')
  else:
    wav=wav[:5*sr]
  spec=librosa.feature.melspectrogram(wav, sr=sr, n_fft=n_fft,
              hop_length=hop_length,n_mels=n_mels,fmin=fmin,fmax=fmax)
  spec_db=librosa.power_to_db(spec,top_db=top_db)
  return spec_db

def spec_to_image(spec, eps=1e-6):
  mean = spec.mean()
  std = spec.std()
  spec_norm = (spec - mean) / (std + eps)  # z-score normalization
  spec_min, spec_max = spec_norm.min(), spec_norm.max()
  spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min) # scaled to be between 0 and 255
  spec_scaled = spec_scaled.astype(np.uint8)
  return spec_scaled

def GetImageFromAudioFile(filepath):
  spec_db = get_melspectrogram_db(filepath)
  image = spec_to_image(spec_db)
  return image


class Model1(Dataset):
    def __init__(self, csv_path, file_path, rows_to_include):
        csvData = pd.read_csv(csv_path)
        csvData = shuffle(csvData)
        self.file_names = []
        self.labels = []
        
        
        for i in range(len(csvData)):
            if(i in rows_to_include):
                self.file_names.append(csvData.iloc[i, 0]) # First column is key which is path of audio file
                self.labels.append(csvData.iloc[i, 3]) # 3rd column (starting at 0) is label
        self.file_path = file_path
        
    def __getitem__(self, index):
        path = self.file_names[index]
                
        # Option 1, waveform
        #sound, sample_rate = torchaudio.load(path, out = None, normalization = True)
        #soundData = sound[0]        
        
        #option2 : mel-specgram
        #sound, sample_rate = torchaudio.load(path, out = None, normalization = True)
        #mel_specgram = torchaudio.transforms.Spectrogram()(sound)
        #soundData = mel_specgram
        #print(soundData.size())
        
        #option3: convert mel-specgram to image
        #sound, sample_rate = torchaudio.load(path, out = None, normalization = True)
        #mel_specgram = torchaudio.transforms.Spectrogram()(sound)
        #mel_specgram = spec_to_image(mel_specgram)
        #soundData = mel_specgram
        
        # Modify here to get different types of Audio processing.
        soundFormatted = GetImageFromAudioFile(path)
        soundFormatted = torch.from_numpy(soundFormatted)
        soundFormatted = soundFormatted.reshape(-1, 128, 157)
        return soundFormatted, self.labels[index]
    
    def __len__(self):
        return len(self.file_names)

#csv_path = 'JsonContentInCsv.csv'
#csv_path = 'JsonContentInCsvmultipleword.csv'
csv_path = 'JsonContentInCsvsingleword.csv'
file_path = ''

train_set = Model1(csv_path, file_path, range(1,5501))
test_set = Model1(csv_path, file_path, range(5502,6502))
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu
#kwargs = {'num_workers': 1, 'pin_memory': True}
print(device)


#train_loader = torch.utils.data.DataLoader(train_set, batch_size = 128, shuffle = True, **kwargs)
#test_loader = torch.utils.data.DataLoader(test_set, batch_size = 128, shuffle = True, **kwargs)

train_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_sz, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_sz, shuffle = True, **kwargs)

Train set size: 5500
Test set size: 1000
cpu


In [98]:
# Model inspired from audio classifier tutorial of Pytorch (open in google colab) from 
# https://pytorch.org/tutorials/beginner/audio_classifier_tutorial.html?highlight=audio

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, padding_mode='VALID')
        self.bn1 = nn.BatchNorm2d(20)
        self.pool1 = nn.MaxPool2d(4)
        
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=5, padding_mode='VALID')
        self.bn2 = nn.BatchNorm2d(20)
        self.pool2 = nn.MaxPool2d(4)
        
        # Disable GRU, if you are using ReLU
        #self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        #self.fc = nn.Linear(hidden_dim, output_dim)
        #self.relu = nn.ReLU()
 
        self.lstm = nn.LSTM(input_size = 960, hidden_size =20, num_layers =1, batch_first=True, dropout=0.5)
        self.fc2 = nn.Linear(in_features = 20, out_features =20, bias = True)
        #self.relu = nn.ReLU() Put it here only if you want to learn its parameter
        
        
        # Dense bottleneck layer
        #self.linear = nn.Linear(hidden_size*6 + features.shape[1], lin_size)
        self.dense1 = nn.Linear(in_features = 20, out_features = 1, bias=True)
        self.dense2 = nn.Linear(in_features = 1, out_features = 1, bias=True)
        #self.tanh = torch.tanh()
        self.dropout = nn.Dropout(0.5) # Layer 8: A dropout layer applied to bottleneck layer
        
        
        # Layer 9: Dysarthric  dense layer --> Linear
        #self.dysarthriclayer = nn.Linear(1, 1)
        
        self.avgPool = nn.AvgPool2d(2)
        self.fc1 = nn.Linear(1, 1)
        
    def forward(self, x):
        #print('shape of input x: ', x.shape)
        x = x.permute(0,1,2,3)# Doesn't change anything.
        
        x = self.conv1(x)
        x = self.dropout(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.dropout(x)
        X = self.bn2(x)
        x = F.relu(x)
        x = self.pool2(x)
        
        # GRU layer
        #print('shape1: ', x.shape)
        #h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        #c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        #h0 = Variable(torch.zeros(1, x.size(0), 20)) 
        #c0 = Variable(torch.zeros(1, x.size(0), 20))
        x = x.reshape(1, 4, 48*20)
        #print('shape of input x0: ', x.shape)
        #x, _ = self.gru(x, (h0,c0))
        x, _ = self.lstm(x)
        x = self.dropout(x)
        #x = x[:, -1, :]
        x = x.reshape(4, 20)
        x = self.fc2(x)
        x = F.relu(x)
        
        #print('shape of input x1: ', x.shape)
        # 2 Dense layers
        x = self.dense1(x)
        x = self.dropout(x)
        x = torch.tanh(x)
        
        #print('shape of input x2: ', x.shape)
        x = self.dense2(x)
        x = self.dropout(x)
        x = torch.tanh(x)
        
        # Dysarthric layer
        #print('shape of input x3: ', x.shape)
        #x = self.dysarthriclayer(x)

        #print(x)
        #x = self.avgPool(x)
        #x = x.view(-1, 20*3*batch_sz)
        x = self.fc1(x)
        y_hat = F.log_softmax(x, dim = 0) 
        return y_hat

model = Net()
model.to(device) #Must send the model to gpu, else remaining code picks up stale version of old model which was earlier sent to gpu
#print(model)


Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), padding_mode=VALID)
  (bn1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1), padding_mode=VALID)
  (bn2): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (lstm): LSTM(960, 20, batch_first=True, dropout=0.5)
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (dense1): Linear(in_features=20, out_features=1, bias=True)
  (dense2): Linear(in_features=1, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (avgPool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=1, out_features=1, bias=True)
)

In [103]:
#optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.09)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)

def train(model, epoch):
    #if torch.cuda.is_available():
    #    model.cuda()
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data = data.to(device, dtype=torch.float32)
        target = target.to(device, dtype=torch.float32) # Note that target should be long, why? Investigate
        data = data.requires_grad_() #set requires_grad to True for training
        output = model(data)
        output = output.view(-1,1)
        #target = target.view(-1,1).squeeze_()
        target = target.view(-1,1)
        
        # old code
        #output = output.permute(1, 0, 2) #original output dimensions are batchSize x 1 x number_of_classes
        #print('shape of output after permute', output.shape)
        #loss = F.nll_loss(output[0], target) #the loss functions expects a batchSizex10 input
        
        #print(output.shape)
        #print(target.shape)
        #print(output)
        #print(target)
        
        '''
        #loss = F.nll_loss(output, target) #the loss functions expects a batchSizex10 input
        criterion = nn.CrossEntropyLoss()
        loss = criterion(output, target)
        '''
        
        pos_weight = torch.ones([1])  # All weights are equal to 1
        criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        loss = criterion(output, target)  # -log(sigmoid(1.5))

        #loss_fn.backward()
        optimizer.step()
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))
def test(model, epoch):
    #if torch.cuda.is_available():
    #    model.cuda()
    model.eval()
    correct = 0
    for data, target in test_loader:
        data = data.to(device, dtype=torch.float32)
        target = target.to(device, dtype=torch.long)
        output = model(data)

        _, pred = torch.max(output, 1)
        correct += pred.eq(target).cpu().sum().item()
        
        print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
        
# Actual training of the model
log_interval = 20
#for epoch in range(1, 41):
for epoch in range(1, 41):
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
    scheduler.step()
    train(model, epoch)
    test(model, epoch)

Train Epoch: 1 [0/5500 (0%)]	Loss: 0.221547
Train Epoch: 1 [80/5500 (1%)]	Loss: 0.533135
Train Epoch: 1 [160/5500 (3%)]	Loss: 1.417929
Train Epoch: 1 [240/5500 (4%)]	Loss: 0.532089
Train Epoch: 1 [320/5500 (6%)]	Loss: 0.221540
Train Epoch: 1 [400/5500 (7%)]	Loss: 1.153950
Train Epoch: 1 [480/5500 (9%)]	Loss: 0.485812
Train Epoch: 1 [560/5500 (10%)]	Loss: 0.570218
Train Epoch: 1 [640/5500 (12%)]	Loss: 0.484686
Train Epoch: 1 [720/5500 (13%)]	Loss: 0.569102
Train Epoch: 1 [800/5500 (15%)]	Loss: 1.218411
Train Epoch: 1 [880/5500 (16%)]	Loss: 1.218473
Train Epoch: 1 [960/5500 (17%)]	Loss: 1.040738
Train Epoch: 1 [1040/5500 (19%)]	Loss: 1.041576
Train Epoch: 1 [1120/5500 (20%)]	Loss: 0.952187
Train Epoch: 1 [1200/5500 (22%)]	Loss: 0.846901
Train Epoch: 1 [1280/5500 (23%)]	Loss: 1.149802
Train Epoch: 1 [1360/5500 (25%)]	Loss: 0.220183
Train Epoch: 1 [1440/5500 (26%)]	Loss: 0.220225
Train Epoch: 1 [1520/5500 (28%)]	Loss: 1.660211
Train Epoch: 1 [1600/5500 (29%)]	Loss: 0.221535
Train Epoch: 1 


Test set: Accuracy: 338/1000 (34%)


Test set: Accuracy: 340/1000 (34%)


Test set: Accuracy: 344/1000 (34%)


Test set: Accuracy: 347/1000 (35%)


Test set: Accuracy: 348/1000 (35%)


Test set: Accuracy: 349/1000 (35%)


Test set: Accuracy: 353/1000 (35%)


Test set: Accuracy: 354/1000 (35%)


Test set: Accuracy: 356/1000 (36%)


Test set: Accuracy: 359/1000 (36%)


Test set: Accuracy: 363/1000 (36%)


Test set: Accuracy: 366/1000 (37%)


Test set: Accuracy: 366/1000 (37%)


Test set: Accuracy: 368/1000 (37%)


Test set: Accuracy: 369/1000 (37%)


Test set: Accuracy: 371/1000 (37%)


Test set: Accuracy: 372/1000 (37%)


Test set: Accuracy: 375/1000 (38%)


Test set: Accuracy: 377/1000 (38%)


Test set: Accuracy: 379/1000 (38%)


Test set: Accuracy: 382/1000 (38%)


Test set: Accuracy: 384/1000 (38%)


Test set: Accuracy: 386/1000 (39%)


Test set: Accuracy: 388/1000 (39%)


Test set: Accuracy: 390/1000 (39%)


Test set: Accuracy: 392/1000 (39%)


Test set: Accuracy: 395/1000 (40%)





Test set: Accuracy: 56/1000 (6%)


Test set: Accuracy: 59/1000 (6%)


Test set: Accuracy: 61/1000 (6%)


Test set: Accuracy: 63/1000 (6%)


Test set: Accuracy: 65/1000 (6%)


Test set: Accuracy: 68/1000 (7%)


Test set: Accuracy: 68/1000 (7%)


Test set: Accuracy: 72/1000 (7%)


Test set: Accuracy: 73/1000 (7%)


Test set: Accuracy: 76/1000 (8%)


Test set: Accuracy: 80/1000 (8%)


Test set: Accuracy: 82/1000 (8%)


Test set: Accuracy: 85/1000 (8%)


Test set: Accuracy: 88/1000 (9%)


Test set: Accuracy: 91/1000 (9%)


Test set: Accuracy: 94/1000 (9%)


Test set: Accuracy: 97/1000 (10%)


Test set: Accuracy: 99/1000 (10%)


Test set: Accuracy: 101/1000 (10%)


Test set: Accuracy: 103/1000 (10%)


Test set: Accuracy: 105/1000 (10%)


Test set: Accuracy: 108/1000 (11%)


Test set: Accuracy: 111/1000 (11%)


Test set: Accuracy: 114/1000 (11%)


Test set: Accuracy: 116/1000 (12%)


Test set: Accuracy: 120/1000 (12%)


Test set: Accuracy: 122/1000 (12%)


Test set: Accuracy: 124/1000 (12%)


Test set: Accuracy: 613/1000 (61%)


Test set: Accuracy: 616/1000 (62%)


Test set: Accuracy: 617/1000 (62%)


Test set: Accuracy: 620/1000 (62%)


Test set: Accuracy: 622/1000 (62%)

Train Epoch: 3 [0/5500 (0%)]	Loss: 0.846718
Train Epoch: 3 [80/5500 (1%)]	Loss: 1.350371
Train Epoch: 3 [160/5500 (3%)]	Loss: 1.459544
Train Epoch: 3 [240/5500 (4%)]	Loss: 0.530679
Train Epoch: 3 [320/5500 (6%)]	Loss: 0.223144
Train Epoch: 3 [400/5500 (7%)]	Loss: 0.839838
Train Epoch: 3 [480/5500 (9%)]	Loss: 0.530820
Train Epoch: 3 [560/5500 (10%)]	Loss: 0.220238
Train Epoch: 3 [640/5500 (12%)]	Loss: 0.221542
Train Epoch: 3 [720/5500 (13%)]	Loss: 1.041647
Train Epoch: 3 [800/5500 (15%)]	Loss: 1.459545
Train Epoch: 3 [880/5500 (16%)]	Loss: 0.220044
Train Epoch: 3 [960/5500 (17%)]	Loss: 1.046378
Train Epoch: 3 [1040/5500 (19%)]	Loss: 0.846901
Train Epoch: 3 [1120/5500 (20%)]	Loss: 1.047825
Train Epoch: 3 [1200/5500 (22%)]	Loss: 0.633211
Train Epoch: 3 [1280/5500 (23%)]	Loss: 1.041616
Train Epoch: 3 [1360/5


Test set: Accuracy: 314/1000 (31%)


Test set: Accuracy: 317/1000 (32%)


Test set: Accuracy: 321/1000 (32%)


Test set: Accuracy: 324/1000 (32%)


Test set: Accuracy: 326/1000 (33%)


Test set: Accuracy: 330/1000 (33%)


Test set: Accuracy: 332/1000 (33%)


Test set: Accuracy: 334/1000 (33%)


Test set: Accuracy: 336/1000 (34%)


Test set: Accuracy: 339/1000 (34%)


Test set: Accuracy: 340/1000 (34%)


Test set: Accuracy: 342/1000 (34%)


Test set: Accuracy: 346/1000 (35%)


Test set: Accuracy: 348/1000 (35%)


Test set: Accuracy: 351/1000 (35%)


Test set: Accuracy: 354/1000 (35%)


Test set: Accuracy: 357/1000 (36%)


Test set: Accuracy: 361/1000 (36%)


Test set: Accuracy: 362/1000 (36%)


Test set: Accuracy: 365/1000 (36%)


Test set: Accuracy: 367/1000 (37%)


Test set: Accuracy: 369/1000 (37%)


Test set: Accuracy: 371/1000 (37%)


Test set: Accuracy: 373/1000 (37%)


Test set: Accuracy: 376/1000 (38%)


Test set: Accuracy: 379/1000 (38%)


Test set: Accuracy: 382/1000 (38%)




...


Test set: Accuracy: 609/1000 (61%)


Test set: Accuracy: 611/1000 (61%)


Test set: Accuracy: 614/1000 (61%)


Test set: Accuracy: 615/1000 (62%)


Test set: Accuracy: 619/1000 (62%)


Test set: Accuracy: 622/1000 (62%)

Train Epoch: 38 [0/5500 (0%)]	Loss: 0.952187
Train Epoch: 38 [80/5500 (1%)]	Loss: 0.951758
Train Epoch: 38 [160/5500 (3%)]	Loss: 1.418632
Train Epoch: 38 [240/5500 (4%)]	Loss: 1.218553
Train Epoch: 38 [320/5500 (6%)]	Loss: 0.916291
Train Epoch: 38 [400/5500 (7%)]	Loss: 0.731019
Train Epoch: 38 [480/5500 (9%)]	Loss: 1.218418
Train Epoch: 38 [560/5500 (10%)]	Loss: 0.951650
Train Epoch: 38 [640/5500 (12%)]	Loss: 1.262864
Train Epoch: 38 [720/5500 (13%)]	Loss: 0.685962
Train Epoch: 38 [800/5500 (15%)]	Loss: 0.951728
Train Epoch: 38 [880/5500 (16%)]	Loss: 0.685962
Train Epoch: 38 [960/5500 (17%)]	Loss: 0.685962
Train Epoch: 38 [1040/5500 (19%)]	Loss: 0.570208
Train Epoch: 38 [1120/5500 (20%)]	Loss: 0.220262
Train Epoch: 38 [1200/5500 (22%)]	Loss: 0.220044
Train Epoch: 38 [


Test set: Accuracy: 327/1000 (33%)


Test set: Accuracy: 330/1000 (33%)


Test set: Accuracy: 333/1000 (33%)


Test set: Accuracy: 334/1000 (33%)


Test set: Accuracy: 334/1000 (33%)


Test set: Accuracy: 337/1000 (34%)


Test set: Accuracy: 339/1000 (34%)


Test set: Accuracy: 341/1000 (34%)


Test set: Accuracy: 343/1000 (34%)


Test set: Accuracy: 344/1000 (34%)


Test set: Accuracy: 348/1000 (35%)


Test set: Accuracy: 350/1000 (35%)


Test set: Accuracy: 352/1000 (35%)


Test set: Accuracy: 353/1000 (35%)


Test set: Accuracy: 355/1000 (36%)


Test set: Accuracy: 357/1000 (36%)


Test set: Accuracy: 360/1000 (36%)


Test set: Accuracy: 361/1000 (36%)


Test set: Accuracy: 363/1000 (36%)


Test set: Accuracy: 366/1000 (37%)


Test set: Accuracy: 369/1000 (37%)


Test set: Accuracy: 371/1000 (37%)


Test set: Accuracy: 373/1000 (37%)


Test set: Accuracy: 376/1000 (38%)


Test set: Accuracy: 377/1000 (38%)


Test set: Accuracy: 378/1000 (38%)


Test set: Accuracy: 380/1000 (38%)





Test set: Accuracy: 38/1000 (4%)


Test set: Accuracy: 40/1000 (4%)


Test set: Accuracy: 41/1000 (4%)


Test set: Accuracy: 43/1000 (4%)


Test set: Accuracy: 47/1000 (5%)


Test set: Accuracy: 50/1000 (5%)


Test set: Accuracy: 50/1000 (5%)


Test set: Accuracy: 53/1000 (5%)


Test set: Accuracy: 54/1000 (5%)


Test set: Accuracy: 58/1000 (6%)


Test set: Accuracy: 60/1000 (6%)


Test set: Accuracy: 64/1000 (6%)


Test set: Accuracy: 64/1000 (6%)


Test set: Accuracy: 66/1000 (7%)


Test set: Accuracy: 69/1000 (7%)


Test set: Accuracy: 72/1000 (7%)


Test set: Accuracy: 74/1000 (7%)


Test set: Accuracy: 75/1000 (8%)


Test set: Accuracy: 77/1000 (8%)


Test set: Accuracy: 78/1000 (8%)


Test set: Accuracy: 80/1000 (8%)


Test set: Accuracy: 84/1000 (8%)


Test set: Accuracy: 86/1000 (9%)


Test set: Accuracy: 88/1000 (9%)


Test set: Accuracy: 91/1000 (9%)


Test set: Accuracy: 93/1000 (9%)


Test set: Accuracy: 96/1000 (10%)


Test set: Accuracy: 99/1000 (10%)


Test set: Accurac


Test set: Accuracy: 590/1000 (59%)


Test set: Accuracy: 593/1000 (59%)


Test set: Accuracy: 595/1000 (60%)


Test set: Accuracy: 599/1000 (60%)


Test set: Accuracy: 600/1000 (60%)


Test set: Accuracy: 603/1000 (60%)


Test set: Accuracy: 607/1000 (61%)


Test set: Accuracy: 608/1000 (61%)


Test set: Accuracy: 612/1000 (61%)


Test set: Accuracy: 615/1000 (62%)


Test set: Accuracy: 618/1000 (62%)


Test set: Accuracy: 619/1000 (62%)


Test set: Accuracy: 622/1000 (62%)

Train Epoch: 40 [0/5500 (0%)]	Loss: 0.531383
Train Epoch: 40 [80/5500 (1%)]	Loss: 0.220230
Train Epoch: 40 [160/5500 (3%)]	Loss: 0.686205
Train Epoch: 40 [240/5500 (4%)]	Loss: 0.839306
Train Epoch: 40 [320/5500 (6%)]	Loss: 0.916291
Train Epoch: 40 [400/5500 (7%)]	Loss: 1.047149
Train Epoch: 40 [480/5500 (9%)]	Loss: 1.048801
Train Epoch: 40 [560/5500 (10%)]	Loss: 0.846901
Train Epoch: 40 [640/5500 (12%)]	Loss: 1.047240
Train Epoch: 40 [720/5500 (13%)]	Loss: 1.673235
Train Epoch: 40 [800/5500 (15%)]	Loss: 0.915513
T


Test set: Accuracy: 306/1000 (31%)


Test set: Accuracy: 309/1000 (31%)


Test set: Accuracy: 312/1000 (31%)


Test set: Accuracy: 314/1000 (31%)


Test set: Accuracy: 316/1000 (32%)


Test set: Accuracy: 318/1000 (32%)


Test set: Accuracy: 321/1000 (32%)


Test set: Accuracy: 323/1000 (32%)


Test set: Accuracy: 326/1000 (33%)


Test set: Accuracy: 328/1000 (33%)


Test set: Accuracy: 332/1000 (33%)


Test set: Accuracy: 333/1000 (33%)


Test set: Accuracy: 336/1000 (34%)


Test set: Accuracy: 338/1000 (34%)


Test set: Accuracy: 342/1000 (34%)


Test set: Accuracy: 346/1000 (35%)


Test set: Accuracy: 348/1000 (35%)


Test set: Accuracy: 350/1000 (35%)


Test set: Accuracy: 353/1000 (35%)


Test set: Accuracy: 355/1000 (36%)


Test set: Accuracy: 355/1000 (36%)


Test set: Accuracy: 359/1000 (36%)


Test set: Accuracy: 361/1000 (36%)


Test set: Accuracy: 364/1000 (36%)


Test set: Accuracy: 367/1000 (37%)


Test set: Accuracy: 370/1000 (37%)


Test set: Accuracy: 373/1000 (37%)




In [ ]:
x = torch.empty(4, 20, 3, 4)
x = x.reshape(4, 20, 3*4)
x.shape